In [ ]:
%pip install langchain langchain-community langchain_ollama langchain_pinecone

In [1]:
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM

# Configure Ollama LLM
ollama_llm = OllamaLLM(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbeddings(
    model="nomic-embed-text:latest",  # Correct parameter name is `model`
    base_url="http://localhost:11434",  # Base URL for the Ollama service
)

In [2]:
from pinecone import Pinecone, ServerlessSpec
import getpass
import os
import time

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [3]:
import time

index_name = "sample"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [4]:
pc.describe_index(index_name)

{
    "name": "sample",
    "dimension": 768,
    "metric": "cosine",
    "host": "sample-zhmtpmp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}

In [5]:
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]



In [6]:
vectorstore = PineconeVectorStore.from_documents(
    docs, index_name=index_name, embedding=ollama_embedding
)

In [7]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [8]:
result = retriever.invoke("dinosaur")
result

(Document(id='0202e997-2ed1-40bc-8aef-a738ab06028f', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993.0, 'score': 0.701512575}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(id='4c01501a-50c0-497b-b2cc-1ec5c7fd884a', metadata={'genre': 'animated', 'year': 1995.0, 'score': 0.541614771}, page_content='Toys come alive and have a blast doing so'),
 Document(id='f979ba39-2624-4783-a057-35980ed6ecae', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979.0, 'score': 0.420702457}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(id='ba4afa98-27f1-4ec6-8450-2466fe87bda2', metadata={'director': 'Christopher Nolan', 'rating': 8.2, 'year': 2010.0, 'score': 0.417136729}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'))